# Option 1: Make your own base container from docker

### Important: Base containers should be approved before use.

### Building and registering the container

The following shell code shows how to build the container image using `docker build` and push the container image to ECR using `docker push`. 

This code looks for an ECR repository in the account you're using and the current region. If the repository doesn't exist, the script will create it.

### Algorithm name as environment viariable


In [ ]:
algorithm_name="ubuntu-16.04-base-with-pip"
%env algorithm_name=$algorithm_name

### Get account info for Dockerfile


In [ ]:
get_account=!aws sts get-caller-identity --query Account --output text --endpoint-url https://sts.us-east-2.amazonaws.com
account = get_account[0]
%env account=$account

get_region=!aws configure get region
region = get_region[0]
%env region=$region

fullname=f"{account}.dkr.ecr.{region}.amazonaws.com/{algorithm_name}:latest"
%env fullname=$fullname

### Create ECR repository


In [ ]:
%%sh

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

echo "completed"

### Print out created ECR

In [ ]:
!aws ecr describe-repositories --repository-names "$algorithm_name"

### Build the Docker Container

Before we start, run the below command to take a look at the Dockerfile.

In [ ]:
!cat base_container_docker/Dockerfile

### The below code will login to ECR and then build the docker image and push to ECR.

In [ ]:
%%sh

cd base_container_docker

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region $region --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -t $algorithm_name .
docker tag $algorithm_name $fullname

docker push $fullname

### Print out docker image


In [ ]:
!docker images $algorithm_name

# Option 2: Copy your own base container from AWS ECR

### Important: Base containers should be approved before use.

The below command shows the amazonlinux images already in ECR

In [ ]:
!aws ecr list-images --region us-east-2 --registry-id 137112412989 --repository-name amazonlinux

#### The below code is the same as in option 1 but with a single change in the Dockerfile. 

### Building and registering the container

The following shell code shows how to build the container image using `docker build` and push the container image to ECR using `docker push`. 

This code looks for an ECR repository in the account you're using and the current region. If the repository doesn't exist, the script will create it.

### Algorithm name as environment viariable


In [ ]:
algorithm_name="amazonlinux-base-with-pip-test"
%env algorithm_name=$algorithm_name

### Get account info for Dockerfile

In [ ]:
get_account=!aws sts get-caller-identity --query Account --output text --endpoint-url https://sts.us-east-2.amazonaws.com
account = get_account[0]
%env account=$account

get_region=!aws configure get region
region = get_region[0]
%env region=$region

fullname=f"{account}.dkr.ecr.{region}.amazonaws.com/{algorithm_name}:latest"
%env fullname=$fullname

### Create ECR repository


In [ ]:
%%sh

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

echo "completed"

### Print out created ECR

In [ ]:
!aws ecr describe-repositories --repository-names "$algorithm_name"

### Build the Docker Container

In [ ]:
!cat base_container_ecr/Dockerfile

### The below code will login to ECR and then build the docker image and push to ECR.


In [ ]:
%%sh

cd base_container_ecr

# Get the login command from ECR and execute it directly
# $(aws ecr get-login --region $region --no-include-email)

aws ecr get-login-password --region $region | docker login --username AWS --password-stdin 137112412989.dkr.ecr.$region.amazonaws.com

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -t $algorithm_name .
docker tag $algorithm_name $fullname

docker push $fullname

### Print out docker image


In [ ]:
!docker images $algorithm_name